In [1]:
import pandas as pd
import numpy as np

In [2]:
# Notebook avec un modèle spacy plus efficace
import spacy
nlp = spacy.load("fr_core_news_sm")

In [3]:
import string
ponctuations = string.punctuation + string.octdigits +'’«»'

In [4]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french') + ['être', 'avoir', 'aller', 'faire', 'ceci', 'cela'])
liste_a_oter = ['article', 'amendement', 'sous-amendement', 'projet', 'loi', 'hémycycle', 'décret', 'ministère', 'élu', 'rapporteure', 
                'madame', 'monsieur', 'mesdames', 'mme', 'messieurs', 'ministre', 'président', 'commission', 'cher', 'tel', 
               'groupe', 'secrétaire', 'collègue', 'gouvernement', 'assemblée', 'rapporteur', 'chers', 'collègues', 'amendements', 'amendement',
               'majorité', 'défavorable', 'favorable', 'avis', 'député', 'qu’', '-ce', 'lorsque', 'parce', 'quel', 'quoi', 'jusqu’', 'dès', 'puisqu’', 
               'mais', 'donc', 'car', 'tout', 'plus', 'moins', 'bien', 'mal', 'lors', 'surtout', 'pourtant', 'selon', 'cependant', 'grâce', 'sinon', 
               'comme', 'très', 'déjà', 'celui', 'aussi', 'alors', 'afin', 'ainsi', 'combien', 'bis', 'fur', 'interrog', 'avai', 'façon', 'quell',
               'un', 'deux', 'trois', 'quatre', 'cinq', 'six', 'sept', 'huit', 'neuf', 'dix', 'vingt', 'cent', 'tant', 'celui-ci', 'fois', 'seulement',
               'sous', 'sur', 'au-delà', 'deuxième', 'lui-même', 'elle-même']

In [5]:
def lemmatize(chaine):
    liste = []
    for token in nlp(chaine):
        if str(token) in ponctuations:
            continue
        liste.append(token.lemma_.lower())
    return liste
def clean_stopwords(liste):
    liste = [w for w in liste if len(w) > 2]
    return [w for w in liste if (not w in stop_words) and (not w in liste_a_oter)]
def clean_numbers(liste):
    for nb in string.octdigits:
        liste = [w for w in liste if nb not in w]
    return liste

In [6]:
interventions = pd.read_csv('data/interventions.csv')
interventions.dropna(inplace=True)

In [7]:
line_out = [' et ', 'député', 'rapporteur', 'groupe', 'ministre', 'secrétaire', 'président', ',']
for element in line_out:
    interventions = interventions[interventions.nom.map(lambda x : element not in x)]

In [8]:
def calc_longueur(chaine):
    return len(chaine.split())
interventions['longueur'] = interventions.texte.map(calc_longueur)

In [9]:
mps = pd.read_csv('data/current_mps.csv', sep=';')
interventions_mps = interventions.merge(mps, on='nom', how='right')[['nom',
                                                                 'texte',
                                                                 'longueur']]
len(interventions_mps.nom.unique())

570

In [10]:
interventions_mps.texte = interventions_mps.texte.astype(str)
interventions_mps['clean_texte'] = interventions_mps['texte']

In [11]:
interventions_mps['clean_texte'] = interventions_mps['clean_texte'].map(lemmatize)

In [12]:
interventions_mps['clean_texte'] = interventions_mps['clean_texte'].map(clean_stopwords)

In [13]:
interventions_mps['clean_texte'] = interventions_mps['clean_texte'].map(clean_numbers)

In [14]:
interventions_mps

,nom,texte,longueur,clean_texte
0,Cédric Roussel,L’article 4 a pour objectif de fournir au CSA ...,76.0,"[objectif, fournir, csa, nouveau, motif, refus..."
1,Cédric Roussel,Il est ici question des certificats de qualifi...,139.0,"[ici, question, certificat, qualification, pro..."
2,Cédric Roussel,Son objet est de rendre les certificats de qua...,33.0,"[objet, rendre, certificat, qualification, pro..."
3,Cédric Roussel,L’article 13 du présent projet de loi vise à r...,40.0,"[présent, vis, remplacer, période, professionn..."
4,Cédric Roussel,C’est faux !,3.0,[faux]
...,...,...,...,...
339623,Victor Habert-Dassault,"Le régiment du service militaire adapté, le RS...",80.0,"[régiment, service, militaire, adapter, rsma, ..."
339624,Victor Habert-Dassault,La mission,2.0,[mission]
339625,Victor Habert-Dassault,"Dans ce contexte, le groupe Les Républicains, ...",29.0,"[contexte, républicain, opposer, feuille, rout..."
339626,Victor Habert-Dassault,La loi relative à la croissance et la transfor...,55.0,"[relatif, croissance, transformation, entrepri..."


In [15]:
words_mps = {}
words_mps['nom'] = []
words_mps['mots'] = []
for nom in interventions_mps['nom'].unique():
    words_mp = interventions_mps[interventions_mps['nom'] == nom][['clean_texte']]
    words = []
    for texte in words_mp['clean_texte']:
        words += texte
    words_mps['nom'].append(nom)
    words_mps['mots'].append(words)
mots_mps = pd.DataFrame(words_mps)

In [16]:
def get_number_of_occurences(liste):
    occurences = {}
    for s in liste:
        for w in s:
            if w not in occurences:
                occurences[w] = 0
            occurences[w] += 1
    return occurences
occurences = get_number_of_occurences(mots_mps['mots'])
len(occurences)

49238

In [17]:
nb_max = 50000
occ_sup = [k for k,v in occurences.items() if v >= nb_max]
print(len(occ_sup))
print(occ_sup)

1
['pouvoir']


In [18]:
nb_min = 10
occ_inf = [k for k,v in occurences.items() if v <= nb_min]
print(len(occ_inf))
print(occ_inf)

34783
['conventionnemer', 'hertzien', 'cqp', 'élaboré', 'prometteuse', 'bêtemer', 'galderma', 'nestlé', 'sophia', 'antipolis', 'technopole', 'valetta', 'ardisson', 'pré-instruction', 'nba', 'symbiose', 'unificateur', 'plébiscite', 'protagoniste', 'byod', 'inciviliter', 'tapage', 'cyberhaine', 'cyberviolence', 'matérialise', 'immobiliers', 'opci', 'conciergerie', 'bayeux', 'lefèvre', 'encher', 'motiv', 'ambitionne', 'mouammar', 'kadhafi', 'exacerbation', 'préélémentaire', 'socioéconomique', 'turnover', 'affrontent', 'engageaient', 'printanier', 'épilogue', 'braillard', 'sympathisant', 'kaboul', 'cycl', 'visionnage', 'locomotif', 'supportérisme', 'houlier', 'bas-relief', 'lemot', 'eim', 'mann', 'libéron', 'bâtit', 'weinstein', 'admiratifs', 'titanesque', 'kosovo', 'ralliement', 'sarthois', 'arjowiggins', 'sequana', 'rima', 'laycuras', 'barkhan', 'hcr', 'vaillamment', 'plagiat', 'name', 'shame', 'tué', 'discothécaire', 'finess', 'élastique', 'bessé-sur-braye', 'chrd', 'savourer', 'dynamis

In [19]:
def keep_words(X, occurences, max_occurence, min_occurence):
    kept_words = {k:v for k, v in occurences.items() if v >= min_occurence and v <= max_occurence}
    output_sentences = [[word for word in sentence if word in kept_words] for sentence in X]    
    return output_sentences

In [20]:
mots_mps['clean_mots'] = keep_words(mots_mps['mots'], occurences, max_occurence=nb_max, min_occurence=nb_min)

In [21]:
mots_mps

,nom,mots,clean_mots
0,Cédric Roussel,"[objectif, fournir, csa, nouveau, motif, refus...","[objectif, fournir, csa, nouveau, motif, refus..."
1,Pascale Fontenel-Personne,"[éducation, national, juillet, annoncer, créat...","[éducation, national, juillet, annoncer, créat..."
2,Guillaume Kasbarian,"[question, adresse, transport, associer, auror...","[question, adresse, transport, associer, auror..."
3,Cyrille Isaac-Sibille,"[vouloir, commencer, propos, rendre, hommage, ...","[vouloir, commencer, propos, rendre, hommage, ..."
4,Guillaume Vuilletet,"[résumer, fin, intervention, principe, général...","[résumer, fin, intervention, principe, général..."
...,...,...,...
565,Luc Lamirault,"[remercier, tenir, avant, dire, fierté, nouvea...","[remercier, tenir, avant, dire, fierté, nouvea..."
566,Christelle Petex-Levet,"[remercier, mot, accueil, grand, honneur, repr...","[remercier, mot, accueil, grand, honneur, repr..."
567,Lamia El Aaraje,"[débat, apprêter, conclure, parfois, passionne...","[débat, apprêter, conclure, parfois, passionne..."
568,Sophie Métadier,"[état, charger, personne, handicaper, institut...","[état, charger, personne, handicaper, institut..."


In [22]:
def most_frequent_words(liste):
    frequent_words = {}
    list_words = []
    for word in liste:
        if word not in list_words:
            frequent_words[word] = 0
            list_words.append(word)
        frequent_words[word] += 1    
    return frequent_words
def list_frequent_words(dico):
    list_words = []
    for key in dico.keys():
        list_words.append(key)
    list_words.sort(key=lambda elt: dico[elt], reverse=True)
    return list_words

In [23]:
mots_mps['nbre_mots'] = mots_mps['mots'].map(len)
mots_mps['freq_mots'] = mots_mps['clean_mots'].map(most_frequent_words)
mots_mps['liste_mots'] = mots_mps['freq_mots'].map(list_frequent_words)
mots_mps['nbre_freq_mots'] = mots_mps['freq_mots'].map(len)
mots_mps.drop(columns='clean_mots', inplace=True)
mots_mps

,nom,mots,nbre_mots,freq_mots,liste_mots,nbre_freq_mots
0,Cédric Roussel,"[objectif, fournir, csa, nouveau, motif, refus...",2754,"{'objectif': 4, 'fournir': 2, 'csa': 2, 'nouve...","[sport, sportif, euro, notamment, vise, entrep...",1331
1,Pascale Fontenel-Personne,"[éducation, national, juillet, annoncer, créat...",1393,"{'éducation': 2, 'national': 5, 'juillet': 1, ...","[économie, france, question, état, devoir, éga...",821
2,Guillaume Kasbarian,"[question, adresse, transport, associer, auror...",11948,"{'question': 58, 'adresse': 5, 'transport': 4,...","[public, permettre, procédure, dire, question,...",2943
3,Cyrille Isaac-Sibille,"[vouloir, commencer, propos, rendre, hommage, ...",9935,"{'vouloir': 35, 'commencer': 9, 'propos': 6, '...","[santé, médecin, devoir, permettre, public, mé...",2620
4,Guillaume Vuilletet,"[résumer, fin, intervention, principe, général...",7133,"{'résumer': 1, 'fin': 15, 'intervention': 8, '...","[public, dire, devoir, texte, permettre, voulo...",2347
...,...,...,...,...,...,...
565,Luc Lamirault,"[remercier, tenir, avant, dire, fierté, nouvea...",1029,"{'remercier': 1, 'tenir': 2, 'avant': 4, 'dire...","[prix, produit, devoir, crise, agriculteur, se...",630
566,Christelle Petex-Levet,"[remercier, mot, accueil, grand, honneur, repr...",102,"{'remercier': 3, 'mot': 1, 'accueil': 1, 'gran...","[remercier, haute-savoie, tenir, difficulté, p...",91
567,Lamia El Aaraje,"[débat, apprêter, conclure, parfois, passionne...",5722,"{'débat': 26, 'apprêter': 2, 'conclure': 3, 'p...","[sanitaire, devoir, dire, texte, passer, santé...",1950
568,Sophie Métadier,"[état, charger, personne, handicaper, institut...",544,"{'état': 2, 'charger': 3, 'personne': 1, 'hand...","[public, chemin, rural, enfant, considérer, re...",377


In [24]:
mots_mps

,nom,mots,nbre_mots,freq_mots,liste_mots,nbre_freq_mots
0,Cédric Roussel,"[objectif, fournir, csa, nouveau, motif, refus...",2754,"{'objectif': 4, 'fournir': 2, 'csa': 2, 'nouve...","[sport, sportif, euro, notamment, vise, entrep...",1331
1,Pascale Fontenel-Personne,"[éducation, national, juillet, annoncer, créat...",1393,"{'éducation': 2, 'national': 5, 'juillet': 1, ...","[économie, france, question, état, devoir, éga...",821
2,Guillaume Kasbarian,"[question, adresse, transport, associer, auror...",11948,"{'question': 58, 'adresse': 5, 'transport': 4,...","[public, permettre, procédure, dire, question,...",2943
3,Cyrille Isaac-Sibille,"[vouloir, commencer, propos, rendre, hommage, ...",9935,"{'vouloir': 35, 'commencer': 9, 'propos': 6, '...","[santé, médecin, devoir, permettre, public, mé...",2620
4,Guillaume Vuilletet,"[résumer, fin, intervention, principe, général...",7133,"{'résumer': 1, 'fin': 15, 'intervention': 8, '...","[public, dire, devoir, texte, permettre, voulo...",2347
...,...,...,...,...,...,...
565,Luc Lamirault,"[remercier, tenir, avant, dire, fierté, nouvea...",1029,"{'remercier': 1, 'tenir': 2, 'avant': 4, 'dire...","[prix, produit, devoir, crise, agriculteur, se...",630
566,Christelle Petex-Levet,"[remercier, mot, accueil, grand, honneur, repr...",102,"{'remercier': 3, 'mot': 1, 'accueil': 1, 'gran...","[remercier, haute-savoie, tenir, difficulté, p...",91
567,Lamia El Aaraje,"[débat, apprêter, conclure, parfois, passionne...",5722,"{'débat': 26, 'apprêter': 2, 'conclure': 3, 'p...","[sanitaire, devoir, dire, texte, passer, santé...",1950
568,Sophie Métadier,"[état, charger, personne, handicaper, institut...",544,"{'état': 2, 'charger': 3, 'personne': 1, 'hand...","[public, chemin, rural, enfant, considérer, re...",377


In [25]:
depute = interventions.groupby('nom')
interventions_mps = depute.count()
interventions_mps.drop(columns='longueur', inplace=True)
interventions_mps['longueur'] = depute['longueur'].mean()
interventions_mps

,texte,longueur
nom,,
Adrien Morenas,49,41.673469
Adrien Quatennens,1246,40.715891
Adrien Taquet,2091,39.454328
Agnès Buzyn,2890,42.942215
Agnès Firmin Le Bodo,739,49.637348
...,...,...
Éric Girardin,95,42.810526
Éric Pauget,490,47.936735
Éric Poulliat,236,45.953390


In [26]:
interventions_mps['nom'] = interventions_mps.index
interventions_mps.reset_index(inplace=True, drop=True)

In [27]:
mps = pd.read_csv('data/current_mps.csv', sep=';')
def nettoyage(val):
    if np.isnan(val):
        val = 0
    return val
interventions_mps = interventions_mps.merge(mps, on='nom', how='inner')[['nom', 'texte', 'longueur', 'sexe', 'num_deptmt', 'parti_ratt_financier', 'id_an']]
interventions_mps.longueur = interventions_mps.longueur.map(nettoyage)
interventions_mps.sort_values(by='texte', inplace=True)
interventions_mps.reset_index(inplace=True, drop=True)
interventions_mps.rename(columns={'texte':'nbre_interv', 'longueur': 'mean_length'}, inplace=True)
interv_mps = interventions_mps.merge(mots_mps, on='nom', how='left')
interv_mps

,nom,nbre_interv,mean_length,sexe,num_deptmt,parti_ratt_financier,id_an,mots,nbre_mots,freq_mots,liste_mots,nbre_freq_mots
0,Emmanuel Blairy,1,56.000000,H,62,NaN,720668,"[permettre, dire, honorer, siéger, hémicycle, ...",26,"{'permettre': 1, 'dire': 1, 'honorer': 1, 'sié...","[permettre, dire, honorer, siéger, hémicycle, ...",24
1,Christophe Leclercq,2,30.000000,H,62,La République en Marche,784314,"[enfant, situation, handicap, scolariser, mili...",28,"{'enfant': 2, 'situation': 1, 'handicap': 2, '...","[enfant, handicap, situation, scolariser, mili...",26
2,Edith Audibert,2,94.000000,F,83,Les Républicains,721666,"[question, adress, état, charger, tourisme, fr...",94,"{'question': 2, 'adress': 1, 'état': 2, 'charg...","[tourisme, question, état, français, falloir, ...",76
3,Stéphane Vojetta,3,56.000000,H,999,La République en Marche,721170,"[associer, question, représentant, compatriote...",81,"{'associer': 1, 'question': 1, 'représentant':...","[français, france, international, hors, commer...",62
4,Bernard Brochand,3,4.000000,H,06,Les Républicains,223837,"[acte, entrée, maîtriser]",3,"{'acte': 1, 'entrée': 1, 'maîtriser': 1}","[acte, entrée, maîtriser]",3
...,...,...,...,...,...,...,...,...,...,...,...,...
561,Sébastien Jumel,5494,24.994175,H,76,Parti communiste français,722202,"[premier, voici, vent, fraîcheur, censé, dépou...",56685,"{'premier': 130, 'voici': 9, 'vent': 6, 'censé...","[dire, falloir, vouloir, état, territoire, pre...",6627
562,Fabien Di Filippo,6184,14.263745,H,57,Les Républicains,720386,"[vrai, comprend, volonté, france, insoumis, ob...",33983,"{'vrai': 99, 'comprend': 7, 'volonté': 19, 'fr...","[dire, falloir, question, vouloir, oui, devoir...",5351
563,Ugo Bernalicis,6549,33.668346,H,59,La France Insoumise,720430,"[état, vouloir, abord, associer, propos, tenir...",89304,"{'état': 399, 'vouloir': 433, 'abord': 56, 'as...","[dire, falloir, non, savoir, vouloir, autre, d...",7320
564,Pierre Cordier,6607,8.513092,H,08,Les Républicains,718850,"[bravo, juste, oui, dos, collectivité, local, ...",19689,"{'bravo': 62, 'juste': 116, 'oui': 257, 'dos':...","[dire, oui, falloir, question, raison, vrai, b...",3855


In [28]:
interv_mps.to_pickle('data/mots_mps_bis.pickle')

In [32]:
search_list = ['santé', 'solidarité']
def search_mps(dico):
    return all(item in dico.keys() for item in search_list)
search_mps = interv_mps[interv_mps.freq_mots.map(search_mps)][['nom', 'parti_ratt_financier', 'freq_mots', 'liste_mots']]
for keyword in search_list:
    def constr1(dico):
        return dico[keyword]
    def constr2(dico):
        nb_mots = sum([dico[key] for key in dico.keys()])
        return dico[keyword] * 100 / nb_mots
    search_mps[keyword] = search_mps['freq_mots'].map(constr1)
    search_mps[keyword + '_freq'] = search_mps['freq_mots'].map(constr2)
search_mps.drop(columns='freq_mots', inplace=True)
search_mps['total'] = 0
for column in search_list:
    search_mps['total'] += search_mps[column + '_freq']
search_mps.sort_values(by='total', inplace=True, ascending=False)
search_mps.drop(columns='total', inplace=True)
search_mps.head(10)

,nom,parti_ratt_financier,liste_mots,santé,santé_freq,solidarité,solidarité_freq
195,Charlotte Parmentier-Lecocq,La République en Marche,"[travail, santé, service, prévention, proposit...",76,2.764642,1,0.036377
28,Bernard Bouley,Les Républicains,"[santé, travail, entreprise, salarier, prévent...",10,2.028398,1,0.202840
124,Marc Delatte,La République en Marche,"[santé, éthique, solidarité, question, devoir,...",24,1.520913,11,0.697085
19,Thierry Solère,La République en Marche,"[vrai, hémicycle, rassurer, diminution, impôt,...",1,1.098901,1,1.098901
402,Stéphanie Rist,La République en Marche,"[santé, établissement, étudiant, semble, médec...",165,1.939807,7,0.082295
313,Jean-Carles Grelier,Les Républicains,"[santé, hôpital, dire, sans, depuis, médecin, ...",88,1.665090,13,0.245979
68,Michel Fanget,Mouvement Démocrate,"[accord, santé, france, entre, droit, devoir, ...",14,1.484624,3,0.318134
300,Bernard Perrut,Les Républicains,"[santé, établissement, devoir, social, grand, ...",105,1.550044,15,0.221435
411,Cyrille Isaac-Sibille,Mouvement Démocrate,"[santé, médecin, devoir, permettre, public, mé...",151,1.559112,20,0.206505
281,Didier Martin,La République en Marche,"[santé, conseil, public, travail, état, person...",66,1.517241,4,0.091954


In [33]:
parti = interv_mps.groupby('parti_ratt_financier')
interv_parti = parti.sum()
interv_parti['nbre_mps'] = parti['nom'].count()
interv_parti.drop(columns=['mean_length', 'nbre_mots', 'id_an'], inplace=True)
interv_parti['nbre_interv_mp'] = interv_parti['nbre_interv']/interv_parti['nbre_mps']
interv_parti['mean_length'] = parti['mean_length'].mean()
interv_parti['nbre_mots'] = parti['nbre_mots'].mean()
interv_parti

,nbre_interv,nbre_freq_mots,nbre_mps,nbre_interv_mp,mean_length,nbre_mots
parti_ratt_financier,,,,,,
Calédonie Ensemble,470,3953,2,235.000000,56.622431,5810.500000
Cap sur l'avenir,55,852,1,55.000000,57.472727,1463.000000
Debout la France,1121,5481,2,560.500000,50.846614,9805.000000
Europe Écologie Les Verts,6154,29204,12,512.833333,46.306238,9320.333333
La France Insoumise,39674,80922,16,2479.625000,38.709990,36487.125000
La République en Marche,79196,376771,271,292.236162,42.608528,4510.040590
Les Républicains,90468,224477,102,886.941176,36.149954,10270.696078
Mouvement Démocrate,30378,135880,68,446.735294,45.383046,7293.382353
Non déclaré(s),409,2583,2,204.500000,37.154594,2892.000000


In [34]:
sexe = interv_mps.groupby('sexe')
interv_sexe = sexe.sum()
interv_sexe['nbre_mps'] = sexe['nom'].count()
interv_sexe.drop(columns=['mean_length', 'nbre_mots', 'id_an'], inplace=True)
interv_sexe['nbre_interv_mp'] = interv_sexe['nbre_interv']/interv_sexe['nbre_mps']
interv_sexe['mean_length'] = sexe['mean_length'].mean()
interv_sexe['nbre_mots'] = sexe['nbre_mots'].mean()
interv_sexe

,nbre_interv,nbre_freq_mots,nbre_mps,nbre_interv_mp,mean_length,nbre_mots
sexe,,,,,,
F,95631,413602,236,405.216102,45.248591,6797.029661
H,243993,709068,330,739.372727,39.262045,9897.578788
